In [25]:
import pandas as pd
import os
from tqdm import tqdm
import json

In [29]:
dirs = ['biorxiv_medrxiv']
docs = []

for d in dirs:
    for file in tqdm(os.listdir(f"{d}/{d}")):
        file_path = f"{d}/{d}/{file}"
        j = json.load(open(file_path,"rb"))
#         print(j)
        
        paper_id = j['paper_id']
        title = j['metadata']['title']
        try:
            abstract = j['abstract'][0]['text']
        except:
            abstract = ''
            
        full_text = ''
        for text in j['body_text']:
            full_text += text['text']+'\n\n'
        docs.append([paper_id,title,abstract,full_text])
df = pd.DataFrame(docs,columns = ['paper_id','title','abstract','full_text'])

100%|███████████████████████████████████████████████████████████████████████████████| 885/885 [00:01<00:00, 486.40it/s]


In [31]:
df.head()

,paper_id,title,abstract,full_text
0,0015023cc06b5362d332b3baf348d11567ca2fbb,The RNA pseudoknots in foot-and-mouth disease ...,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP..."
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,Healthcare-resource-adjusted vulnerabilities t...,,The 2019-nCoV epidemic has spread across China...
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,"Real-time, MinION-based, amplicon sequencing f...",Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by..."
3,0139ea4ca580af99b602c6435368e7fdbefacb03,A Combined Evidence Approach to Prioritize Nip...,Nipah Virus (NiV) came into limelight recently...,Nipah is an infectious negative-sense single-s...
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Assessing spread risk of Wuhan novel coronavir...,Background: A novel coronavirus (2019-nCoV) em...,"In December 2019, a cluster of patients with p..."


In [72]:
key_words = [' hospital',' personnel',' nurse',' doctor',' aid',
             ' site',' strategies',' strategy',' fund',' equipment']
for word in key_words:
    print(word, df[df['abstract'].str.contains(word)].shape)

 hospital (21, 4)
 personnel (2, 4)
 nurse (2, 4)
 doctor (0, 4)
 aid (7, 4)
 site (31, 4)
 strategies (39, 4)
 strategy (25, 4)
 fund (6, 4)
 equipment (3, 4)
